In [2]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
import pandapower as pp # type: ignore
import matplotlib.pyplot as plt # type: ignore 
import pandapower.plotting as plot


In [3]:
# Crear red
net = pp.create_empty_network()

# Crear barras
barra1 = pp.create_bus(net, vn_kv=110, name="Barra 1")
barra2 = pp.create_bus(net, vn_kv=220, name="Barra 2")
barra1A = pp.create_bus(net, vn_kv=220, name="Barra 1A")
barra1B = pp.create_bus(net, vn_kv=220, name="Barra 1B")
barra2A = pp.create_bus(net, vn_kv=220, name="Barra 2A")
barra2B = pp.create_bus(net, vn_kv=220, name="Barra 2B")
barra3A = pp.create_bus(net, vn_kv=220, name="Barra 3A")

# Crear transformador
pp.create_transformer(net, barra2, barra1, std_type="100 MVA 220/110 kV")

# Crear barra slack
pp.create_ext_grid(net, barra1, vm_pu=1.0, name="Slack bus")

# Crear cargas
pp.create_load(net, barra1A, p_mw=30, q_mvar=20, name="Carga 1A")
pp.create_load(net, barra1B, p_mw=15, q_mvar=10, name="Carga 1B")
pp.create_load(net, barra2A, p_mw=52.5, q_mvar=35, name="Carga 2A")
pp.create_load(net, barra2B, p_mw=90, q_mvar=60, name="Carga 2B")
pp.create_load(net, barra3A, p_mw=22.5, q_mvar=15, name="Carga 3A")

# Crear líneas
pp.create_line(net, barra2, barra1A, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2-1A")
pp.create_line(net, barra1A, barra2A, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1A-2A")
pp.create_line(net, barra2A, barra3A, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2A-3A")
pp.create_line(net, barra3A, barra2B, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L3A-2B")
pp.create_line(net, barra2B, barra1B, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2B-1B")
pp.create_line(net, barra1B, barra2, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1-B2")

# Ejecutar flujo de potencia
pp.runpp(net, algorithm='nr', numba=False, max_iteration=100)

# Obtener el número de iteraciones
iterations = net._ppc["iterations"]

# Mostrar resultados
net.res_line.index = net.line.name
net.res_bus.index = net.bus.name
print(f"Número de iteraciones: {iterations}")

Número de iteraciones: 4


In [4]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
name,,,,
Barra 1,1.000000,0.000000,-212.310191,5.807279
Barra 2,1.033380,-14.276471,0.000000,0.000000
Barra 1A,1.031457,-14.492303,30.000000,20.000000
Barra 1B,1.032375,-14.469768,15.000000,10.000000
Barra 2A,1.029237,-14.730336,52.500000,35.000000
Barra 2B,1.027435,-14.873522,90.000000,60.000000
Barra 3A,1.028365,-14.864945,22.500000,15.000000


In [5]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
name,,,,,,,,,,,,,,
L2-1A,116.806759,-22.385593,-116.542550,2.542957,0.264208,-19.842637,0.302035,0.296588,0.302035,1.033380,-14.276471,1.031457,-14.492303,66.090776
L1A-2A,86.542550,-22.542957,-86.324989,-7.380860,0.217562,-29.923817,0.227536,0.220912,0.227536,1.031457,-14.492303,1.029237,-14.730336,49.789132
L2A-3A,33.824989,-27.619140,-33.778641,-12.542279,0.046347,-40.161419,0.111345,0.091951,0.111345,1.029237,-14.730336,1.028365,-14.864945,24.364315
L3A-2B,11.278641,-2.457721,-11.270332,-27.652178,0.008309,-30.109899,0.029458,0.076272,0.076272,1.028365,-14.864945,1.027435,-14.873522,16.689627
L2B-1B,-78.729668,-32.347822,79.090275,-27.565530,0.360607,-59.913352,0.217407,0.212911,0.217407,1.027435,-14.873522,1.032375,-14.469768,47.572682
L1-B2,-94.090275,17.565530,94.274899,-37.551569,0.184624,-19.986039,0.243312,0.257710,0.257710,1.032375,-14.469768,1.033380,-14.276471,56.391578


In [6]:
net.res_load

,p_mw,q_mvar
0,30.0,20.0
1,15.0,10.0
2,52.5,35.0
3,90.0,60.0
4,22.5,15.0
